<a href="https://colab.research.google.com/github/adarshnitt/Deep-Learning-Models/blob/main/14_fine_tunining_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!nvidia-smi

In [ ]:
# #!pip install -Uqqq pip --progress-bar off
# #!pip install -qqq torch==2.0.1 --progress-bar off
# !pip install -qqq transformers==4.32.1 --progress-bar off
# !pip install -qqq datasets==2.14.4 --progress-bar off
# !pip install -qqq peft==0.5.0 --progress-bar off
# !pip install -qqq bitsandbytes==0.41.1 --progress-bar off
# #!pip install -qqq trl==0.7.1 --progress-bar off

In [ ]:
! pip install peft datasets  bitsandbytes transformers trl

In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [ ]:
! pip install trl
from trl import SFTTrainer # supervised fine tuning trainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

## Data

In [ ]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datas

DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()


def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
{summary}
""".strip()

In [ ]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text) # remove leading trails
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)


def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_text(item["user utterance"])
        text += f"user: {user.strip()}\n"

        agent = clean_text(item["system response"])
        text += f"agent: {agent.strip()}\n"

    return text

In [ ]:
def generate_text(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)

    conversation_text = create_conversation_text(data_point)
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary),
    }

In [ ]:
example = generate_text(dataset["train"][0])

In [ ]:
print(example["summary"])

Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.


In [ ]:
print(example["conversation"])

user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happening please. For example, did it start after an update or after installing a certain app?



In [ ]:
# this is goving to use in my dataset
print(example["text"])

### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.

### Input:
user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
user: My iPhone is on 11.1.2, and my watch is on 4.1.
agent: Thank you. Have you tried restarting both devices since this started happening?
user: I’ve restarted both, also un-paired then re-paired the watch.
agent: Got it. When did you first notice that the two devices were not talking to each other. Do the two devices communicate through other apps such as Messages?
user: Yes, everything seems fine, it’s just Health and activity.
agent: Let’s move to DM and look into this a bit more. When reaching out in DM, let us know when this first started happe

In [ ]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_text)
        .remove_columns(
            [
                "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
            ]
        )
    )

In [ ]:
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])

## Model

In [ ]:
notebook_login()

In [ ]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer
MODEL_NAME = "distilbert/distilgpt2"
model, tokenizer = create_model_and_tokenizer()

In [ ]:
# model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

In [ ]:
# model.config.quantization_config.to_dict()

In [ ]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    #target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

## Training

In [ ]:
OUTPUT_DIR = "experiments"
# #
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs

In [ ]:


training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    #optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,

    max_grad_norm=0.3,
    num_train_epochs=5,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [ ]:
t=dataset["train"].select(range(2))
t

Dataset({
    features: ['conversation', 'summary', 'text'],
    num_rows: 2
})

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"].select(range(2)),
    eval_dataset=dataset["validation"].select(range(2)),
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
dataset["validation"]["text"]

['### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.\n\n### Input:\nuser: Is it normal for my Xbox One X to be making pretty loud fan noises? Even while the console is off? It seems to be louder than my original One.\nagent: Hiya! Just to clarify, does your console have plenty of room for ventilation: Also, is your console plugged directly into a wall outlet?\nuser: Yes it does have plenty of ventilation, and no it is not directly to the wall outlet. I have it connected to a surge protector.\nagent: Thanks for this info, a direct wall outlet connection would be recommended: If you 1  plug the console directly into the wall outlet &amp; test for a bit, do you hear this noise? 2\nuser: It seems to have stopped that noise, thank you! Also does having it connected to a surge protector mess up the console in any way? Because that noise was worrying me\nagent: We could not speculate on this, but a surge protector is not recommended 

In [ ]:
!pip install str2optimizer32bit

ERROR: Could not find a version that satisfies the requirement str2optimizer32bit (from versions: none)
ERROR: No matching distribution found for str2optimizer32bit


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
1,1.001900,3.910430
2,1.013800,3.907279


Checkpoint destination directory experiments/checkpoint-1 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory experiments/checkpoint-2 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2, training_loss=1.0078524947166443, metrics={'train_runtime': 23.2647, 'train_samples_per_second': 0.172, 'train_steps_per_second': 0.086, 'total_flos': 395690803200.0, 'train_loss': 1.0078524947166443, 'epoch': 2.0})

In [ ]:
trainer.save_model("./adarsh_llm/")

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-5): 6 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
          

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

## Inference

In [ ]:
def generate_prompt(
    conversation: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
""".strip()

In [ ]:
examples = []
for data_point in dataset["test"].select(range(5)):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
    conversation = create_conversation_text(data_point)
    examples.append(
        {
            "summary": summary,
            "conversation": conversation,
            "prompt": generate_prompt(conversation),
        }
    )
test_df = pd.DataFrame(examples)
test_df

#### Inference with Base Model

In [ ]:
model, tokenizer = create_model_and_tokenizer()

In [ ]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

#### Example 1

In [ ]:
example = test_df.iloc[0]
print(example.conversation)

In [ ]:
print(example.summary)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary)

#### Example 2

In [ ]:
example = test_df.iloc[1]
print(example.conversation)

In [ ]:
print(example.summary)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary)

#### Example 3

In [ ]:
example = test_df.iloc[2]
print(example.conversation)

In [ ]:
print(example.summary)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary)

### Inference with the Fine-tuned Model

In [ ]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)

#### Example 1

In [ ]:
example = test_df.iloc[0]
pprint(example.summary)

In [ ]:
print(example.conversation)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary)

In [ ]:
pprint(summary.strip().split("\n")[0])

#### Example 2

In [ ]:
example = test_df.iloc[1]
print(example.summary)

In [ ]:
print(example.conversation)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary.strip().split("\n")[0])

#### Example 3

In [ ]:
example = test_df.iloc[2]
print(example.summary)

In [ ]:
print(example.conversation)

In [ ]:
%%time
summary = summarize(model, example.prompt)

In [ ]:
pprint(summary.strip().split("\n")[0])

## References

- https://huggingface.co/datasets/Salesforce/dialogstudio
- https://huggingface.co/meta-llama/Llama-2-7b-hf